In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [44]:
# Loading the processed Data
X_train = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X_train.csv')
X_test = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/X_test.csv')
y_train = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y_train.csv')['Class']
y_test = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/processed/y_test.csv')['Class']

In [45]:
df = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/raw/european_data.csv')

In [46]:
# Separate fraud and legitimate transactions
fraud = df[df['Class'] == 1]  # 492 fraud transactions
legit = df[df['Class'] == 0]  # 284,315 legitimate transactions

# Create training set: all frauds + 2312 legit
np.random.seed(42)  # For reproducibility
legit_train_indices = np.random.choice(legit.index, size=2312, replace=False)
legit_train = legit.loc[legit_train_indices]
train_set = pd.concat([fraud, legit_train])  # Total: 2804 transactions

# Define features (V1-V28 and Amount)
features = ['V' + str(i) for i in range(1, 29)] + ['Amount']

# Extract features and labels for training set
X_train = train_set[features]
y_train = train_set['Class']
amounts_train = train_set['Amount'].values

# Full dataset for final evaluation
X_full = df[features]
y_full = df['Class']
amounts_full = df['Amount'].values

In [47]:
# Standardize features using training set statistics
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_full_scaled = scaler.transform(X_full)  # Apply same transformation to full dataset

In [48]:
# Fitness function
def fitness(solution, X_scaled, y, amounts, r=1):
    weights = solution[:-1]  # First 29 values are weights
    threshold = solution[-1]  # Last value is threshold
    scores = np.dot(X_scaled, weights)  # Compute suspiciousness scores
    predictions = scores >= threshold  # Predict fraud if score >= threshold
    TP = np.logical_and(predictions, y == 1)  # True positives
    FP = np.logical_and(predictions, y == 0)  # False positives
    saving = np.sum(amounts[TP])  # Sum of amounts for TPs
    alerts = np.sum(predictions)  # Total alerts (TP + FP)
    S = saving - r * alerts  # Savings
    return S

In [49]:
# Function to generate a random solution
def generate_solution():
    weights = np.random.rand(29)  # Weights in [0, 1]
    threshold = np.random.uniform(-20, 20)  # Threshold in [-20, 20]
    return np.append(weights, threshold)

# Initialize special solutions
MAX_solution = np.append(np.ones(29), -20)  # High weights, low threshold -> max alerts
MIN_solution = np.append(np.zeros(29), 20)  # Low weights, high threshold -> min alerts

# Initialize population (50 solutions)
population = np.array([MAX_solution, MIN_solution] + [generate_solution() for _ in range(48)])

In [50]:
# GA parameters
max_generations = 300
no_improvement_threshold = 20
best_fitness = -np.inf
counter = 0

for generation in range(max_generations):
    # Generate random weight for recombination
    w = np.random.rand()

    # Generate children via recombination
    children = []
    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            P1 = population[i]
            P2 = population[j]
            child = w * P1 + (1 - w) * P2
            children.append(child)
    children = np.array(children)

    # Mutate one child
    k = np.random.randint(len(children))  # Random child index
    p = np.random.randint(30)  # Random parameter index
    if p < 29:  # Weight mutation
        children[k, p] = np.random.rand()
    else:  # Threshold mutation
        children[k, p] = np.random.uniform(-20, 20)

    # Evaluate fitness for all solutions
    all_solutions = np.vstack([population, children])
    fitness_values = [fitness(sol, X_train_scaled, y_train, amounts_train) for sol in all_solutions]

    # Selection: Keep MAX and MIN, select 48 via roulette wheel
    fitness_values = np.array(fitness_values)
    min_fitness = fitness_values.min()
    adjusted_fitness = fitness_values - min_fitness + 1  # Ensure positive values
    probabilities = adjusted_fitness / adjusted_fitness.sum()
    selected_indices = np.random.choice(len(all_solutions), size=48, p=probabilities, replace=True)
    new_population = np.vstack([all_solutions[0], all_solutions[1], all_solutions[selected_indices]])
    population = new_population

    # Check for improvement
    current_best = max(fitness_values)
    if current_best > best_fitness:
        best_fitness = current_best
        counter = 0
    else:
        counter += 1
    if counter >= no_improvement_threshold:
        print(f"Terminated at generation {generation} due to no improvement")
        break

    # Progress monitoring
    if generation % 10 == 0:
        print(f"Generation {generation}, Best fitness: {current_best}")

Generation 0, Best fitness: 56104.96
Generation 10, Best fitness: 51770.02
Terminated at generation 20 due to no improvement


In [51]:
# Select best solution
final_fitness_values = [fitness(sol, X_train_scaled, y_train, amounts_train) for sol in population]
best_index = np.argmax(final_fitness_values)
best_solution = population[best_index]

In [55]:
# Function to compute detailed metrics
def compute_metrics(solution, X_scaled, y, amounts, r=1):
    weights = solution[:-1]
    threshold = solution[-1]
    scores = np.dot(X_scaled, weights)
    predictions = scores >= threshold
    TP = np.sum(np.logical_and(predictions, y == 1))
    FP = np.sum(np.logical_and(predictions, y == 0))
    FN = np.sum(np.logical_and(~predictions, y == 1))
    TN = np.sum(np.logical_and(~predictions, y == 0))
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    F1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    saving = np.sum(amounts[np.logical_and(predictions, y == 1)])
    alerts = TP + FP
    S = saving - r * alerts
    return {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': F1,
        'Saving': saving,
        'Alerts': alerts,
        'Total Savings (S)': S
    }

# Evaluate on full dataset
metrics = compute_metrics(best_solution, X_full_scaled, y_full, amounts_full)
print("Performance on full dataset:")
for key, value in metrics.items():
    print(f"{key}: {value}")

Performance on full dataset:
TP: 417
FP: 284178
FN: 75
TN: 137
Precision: 0.0014652400780055868
Recall: 0.8475609756097561
F1 Score: 0.0029254227656820547
Saving: 54499.02
Alerts: 284595
Total Savings (S): -1368475.98


In [61]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the full dataset
df = pd.read_csv('/Users/raya/Desktop/fraud-detection/european-dataset/data/raw/european_data.csv')

# Define features and labels
features = ['V' + str(i) for i in range(1, 29)] + ['Amount']
X = df[features]
y = df['Class']
amounts = df['Amount'].values

# Split into training and test sets (80-20) with stratification
X_train_full, X_test, y_train_full, y_test, amounts_train_full, amounts_test = train_test_split(
    X, y, amounts, test_size=0.2, stratify=y, random_state=42
)

# Split training set into sub-training and validation (80-20) with stratification
X_subtrain, X_val, y_subtrain, y_val, amounts_subtrain, amounts_val = train_test_split(
    X_train_full, y_train_full, amounts_train_full, test_size=0.2, stratify=y_train_full, random_state=42
)

# Standardize features using sub-training set statistics
scaler = StandardScaler()
X_subtrain_scaled = scaler.fit_transform(X_subtrain)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Fitness function: maximize F1 score by finding optimal threshold
def fitness(weights, X_scaled, y):
    scores = np.dot(X_scaled, weights)
    thresholds = np.linspace(min(scores), max(scores), 100)
    best_f1 = 0
    for thresh in thresholds:
        predictions = scores >= thresh
        current_f1 = f1_score(y, predictions)
        if current_f1 > best_f1:
            best_f1 = current_f1
    return best_f1

# Generate a random solution (weights only)
def generate_solution():
    return np.random.rand(29)

# Initialize population (50 solutions)
population = np.array([generate_solution() for _ in range(50)])

# GA parameters
max_generations = 100
no_improvement_threshold = 10
best_fitness = -np.inf
counter = 0

# Genetic Algorithm
for generation in range(max_generations):
    # Evaluate fitness on validation set
    fitness_values = [fitness(weights, X_val_scaled, y_val) for weights in population]

    # Selection: roulette wheel
    fitness_values = np.array(fitness_values)
    probabilities = fitness_values / fitness_values.sum()
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities, replace=True)
    selected_population = population[selected_indices]

    # Crossover: uniform crossover for pairs
    children = []
    for i in range(0, len(selected_population), 2):
        if i + 1 < len(selected_population):
            P1 = selected_population[i]
            P2 = selected_population[i + 1]
            mask = np.random.rand(29) < 0.5
            child1 = np.where(mask, P1, P2)
            child2 = np.where(mask, P2, P1)
            children.extend([child1, child2])
        else:
            children.append(selected_population[i])

    # Mutation: 10% chance per child
    mutation_rate = 0.1
    for child in children:
        if np.random.rand() < mutation_rate:
            mutation_index = np.random.randint(29)
            child[mutation_index] = np.random.rand()

    # Replace population
    population = np.array(children)

    # Check for improvement
    current_best = max(fitness_values)
    if current_best > best_fitness:
        best_fitness = current_best
        counter = 0
    else:
        counter += 1
    if counter >= no_improvement_threshold:
        print(f"Terminated at generation {generation} due to no improvement")
        break

    if generation % 10 == 0:
        print(f"Generation {generation}, Best F1 Score: {current_best}")

# Select best weights
final_fitness_values = [fitness(weights, X_val_scaled, y_val) for weights in population]
best_index = np.argmax(final_fitness_values)
best_weights = population[best_index]

# Find optimal threshold on validation set
scores_val = np.dot(X_val_scaled, best_weights)
thresholds = np.linspace(min(scores_val), max(scores_val), 100)
best_f1 = 0
best_threshold = 0
for thresh in thresholds:
    predictions = scores_val >= thresh
    current_f1 = f1_score(y_val, predictions)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = thresh

# Apply to test set
scores_test = np.dot(X_test_scaled, best_weights)
predictions_test = scores_test >= best_threshold

# Compute detailed metrics
def compute_metrics(y_true, y_pred, amounts):
    TP = np.sum(np.logical_and(y_pred, y_true == 1))
    FP = np.sum(np.logical_and(y_pred, y_true == 0))
    FN = np.sum(np.logical_and(~y_pred, y_true == 1))
    TN = np.sum(np.logical_and(~y_pred, y_true == 0))
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return {'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

metrics = compute_metrics(y_test, predictions_test, amounts_test)
print("Test set performance:")
for key, value in metrics.items():
    print(f"{key}: {value}")

Generation 0, Best F1 Score: 0.018779342723004695
Generation 10, Best F1 Score: 0.08695652173913043
Generation 20, Best F1 Score: 0.14
Generation 30, Best F1 Score: 0.16216216216216217
Generation 40, Best F1 Score: 0.17094017094017094
Generation 50, Best F1 Score: 0.20202020202020202
Terminated at generation 60 due to no improvement
Test set performance:
TP: 5
FP: 27
FN: 93
TN: 56837
Precision: 0.15625
Recall: 0.05102040816326531
F1 Score: 0.07692307692307693
